In [1]:
# 线性回归的目标:z=(<x,w> - y)^2 最小
%matplotlib inline
import random
import torch
import pandas as pd
import math
from pymodel.qmodel import *

In [2]:
# 示例:温度为25°C,水的浓度为10g/m^3,气压为101.325kPa(对应标准大气压)
T = 25  # °C
H2O_concentration = 10  # g/m^3
P = 101.325  # kPa

RH = relative_humidity(T, H2O_concentration, P)
print("相对湿度为:", RH, "%")

VPD = vapor_pressure_deficit(T, H2O_concentration, P)
print("蒸汽压差为:", VPD, "kPa")

相对湿度为: 0.5762937775804863 %
蒸汽压差为: 3.067421661187285 kPa


In [3]:
observation_df = pd.read_csv('./data/A_ci.csv')
observation_df = observation_df[observation_df['Photo'] > 0]
observation_df['RH'] = relative_humidity(observation_df['Tair'],observation_df['H2OS'],observation_df['Press'])

Photo = observation_df['Photo'].tolist()
Tleaf = observation_df['Tleaf'].tolist()
CO2S = observation_df['CO2S'].tolist()
RH_S = (observation_df['RH_S']/100).tolist()
PARi = observation_df['PARi'].tolist()
Press = observation_df['Press'].tolist()

labels = torch.tensor(Photo)
features = torch.tensor([Tleaf,CO2S,RH_S,PARi,Press]).T


In [5]:
max(observation_df['RH'])

3.354701840353673

In [ ]:
# 读取数据集

# 用一个生成器来产生小批量的batch
def data_iter(batch_size, features, labels):
    num_examples = len(features)
    indices = list(range(num_examples))
    # 这些样本是随机读取的,没有特定的顺序
    random.shuffle(indices)
    for i in range(0, num_examples, batch_size):
        batch_indices = torch.tensor(
            indices[i: min(i + batch_size, num_examples)])
        yield features[batch_indices], labels[batch_indices]
        
batch_size = 10

for X, y in data_iter(batch_size, features, labels):
    print(X, '\n', y)
    break

tensor([[3.1633e+01, 5.9240e+02, 7.7507e-01, 1.2997e+03, 1.0101e+02],
        [3.2282e+01, 1.9679e+02, 5.4738e-01, 4.9956e+02, 9.9582e+01],
        [3.2342e+01, 1.4857e+03, 7.7074e-01, 1.3011e+03, 9.9820e+01],
        [3.1649e+01, 1.4839e+03, 7.5779e-01, 1.2996e+03, 1.0016e+02],
        [3.0708e+01, 3.8543e+02, 7.4098e-01, 1.2990e+03, 1.0093e+02],
        [2.9673e+01, 1.1892e+03, 6.7103e-01, 5.0013e+02, 1.0111e+02],
        [3.1586e+01, 1.9904e+02, 7.2917e-01, 1.3003e+03, 1.0007e+02],
        [3.3261e+01, 3.9492e+02, 7.6001e-01, 1.2993e+03, 1.0141e+02],
        [3.1212e+01, 5.9619e+02, 7.4939e-01, 1.2990e+03, 1.0064e+02],
        [2.9259e+01, 9.8691e+02, 6.9033e-01, 4.9930e+02, 1.0016e+02]]) 
 tensor([ 5.5442,  1.6842, 11.0676, 13.4320,  3.6117,  8.0323,  0.3271,  4.7600,
         3.3504, 10.2515])


In [ ]:
# 定义模型
leaf = Leaf()
# 定义模型和初始化参数
leaf = Leaf()
initial_params = torch.tensor([0.11, 0.8, 0.001])


def fit_qmodel(X, w):  #@save
    """线性回归模型"""
    result_list = []
    
    for i in range(0,len(X)):
        An_dict = Q_model(leaf = leaf,
                        tleaf= X[i][0],co2 = X[i][1],RH = X[i][2],PAR = X[i][3],patm = X[i][4],
                        cost = w)
        result_list.append(An_dict['An'])

    return torch.tensor(result_list)

# 定义模型和初始化参数
leaf = Leaf()
initial_params = torch.tensor([0.11, 0.8, 0.001])

# 定义损失函数
def loss_fn(y_hat, y):
    return torch.mean((y_hat - y) ** 2)

In [ ]:
# 定义蒙特卡罗采样次数
num_samples = 10

lr = 0.03
batch_size = 100

# 初始化参数估计列表
parameter_estimates = []

initial_params = torch.tensor([0.11, 0.8, 0.001])
best_loss = 1000
best_params = torch.tensor([0.11, 0.8, 0.001])
i = 0
# 进行蒙特卡罗采样和参数估计
while i < 1000:
    # 从参数空间中随机采样一组参数值
    sampled_params = initial_params +  torch.randn_like(initial_params) * lr
    if sum(sampled_params>0)==3:        
        for X, y in data_iter(batch_size, features, labels):
            i+=1
            # 计算模型的预测输出
            y_hat = fit_qmodel(X, sampled_params)
            
            # 计算损失函数值
            loss = loss_fn(y_hat, y)
            if best_loss > loss.item():
                best_loss = loss.item()
                best_params = sampled_params
            # 将参数值和损失函数值添加到列表中
            parameter_estimates.append((sampled_params, loss.item()))
            
            break

# 计算参数的期望值
[best_params,best_loss]


[tensor([0.1093, 0.7897, 0.0500]), 41.18855285644531]

In [ ]:
initial_loss 

37.830787658691406

In [ ]:
fit_qmodel(X, initial_params)

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0])